# SHA3-256

Чтение входного файла

In [1]:
#file = open('input.txt', 'rb')
#file = open('./img/img.png', 'rb')
file = open('./img/large.jpg', 'rb')

BAinput = bytearray(file.read())
print(f'Размер входа: {str(len(BAinput))} байт')
file.close()

Размер входа: 2508587 байт


Константы для вычислений (premutations widths and quataties)

![title](img/img.png)

In [2]:
b = 1600  # размер состояния (5 * 5 * 64)
r = 1088  # rate
l = 6
capacity = b - r
nrounds = 12 + 2 * l  # количество раундов вычислений


Функция дополнения

In [3]:
q = r - (len(BAinput) * 8 % r)
if q == 0:
    BAinput.append(l)
    for i in range(1, r // 8 - 1):
        BAinput.append(0b00000000)
    BAinput.append(0x80)
elif q == 8:
    BAinput.append(l ^ 0x80)
elif q == 16:
    BAinput.append(l)
    BAinput.append(0x80)
else:
    BAinput.append(l)
    for i in range(1, q // 8 - 1):
        BAinput.append(0b00000000)
    BAinput.append(0x80)

Определяем количество блоков

In [4]:
blocksTotal = (len(BAinput) * 8) // r
print(blocksTotal)

18446


Константы раундов и матрица вращений (rotation offsets)

In [5]:
RC = [0 for i in range(24)]
RC[0] = 0x0000000000000001
RC[1] = 0x0000000000008082
RC[2] = 0x800000000000808A
RC[3] = 0x8000000080008000
RC[4] = 0x000000000000808B
RC[5] = 0x0000000080000001
RC[6] = 0x8000000080008081
RC[7] = 0x8000000000008009
RC[8] = 0x000000000000008A
RC[9] = 0x0000000000000088
RC[10] = 0x0000000080008009
RC[11] = 0x000000008000000A
RC[12] = 0x000000008000808B
RC[13] = 0x800000000000008B
RC[14] = 0x8000000000008089
RC[15] = 0x8000000000008003
RC[16] = 0x8000000000008002
RC[17] = 0x8000000000000080
RC[18] = 0x000000000000800A
RC[19] = 0x800000008000000A
RC[20] = 0x8000000080008081
RC[21] = 0x8000000000008080
RC[22] = 0x0000000080000001
RC[23] = 0x8000000080008008

rotat = [[0, 36, 3, 41, 18],
         [1, 44, 10, 45, 2],
         [62, 6, 43, 15, 61],
         [28, 55, 25, 21, 56],
         [27, 20, 39, 8, 14]]

Функция вращения (двигаем переменную a на n бит)

In [6]:
def rot(a, n, total=64):
    b = a << n
    m = 1 << total
    c = b % m
    d = a >> (total - n)
    res = c | d
    return res

# Шаги для функции перестановки

In [7]:
def printLine():
    print("-----------------------")

Шаг тетта

In [8]:
def theta(A):
    C = []
    for x in range(5):
        C.append(A[x][0] ^ A[x][1] ^ A[x][2] ^ A[x][3] ^ A[x][4])
    D = [0, 0, 0, 0, 0]
    for x in range(5):
        D[x] = C[(x - 1) % 5] ^ rot(C[(x + 1) % 5], 1)
    for x in range(5):
        for y in range(5):
            A[x][y] ^= D[x]
    #printLine()
    #print("результат тетта\nA =", A, '\nC =', C)
    #printLine()
    return A, C

Шаг ро и пи

In [9]:
def rho_pi(argsAC):
    A, C = argsAC
    B = []
    for i in range(5):
        B.append([0, 0, 0, 0, 0])
    for x in range(5):
        for y in range(5):
            B[y][(2 * x + 3 * y) % 5] = rot(A[x][y], rotat[x][y])
    #print("Результат ро пи\nA =", A, '\nC =', C, '\nB =', B)
    #printLine()
    return A, B, C

Шаг чи

In [10]:
def chi(argsABC):
    A, B, C = argsABC
    for x in range(5):
        for y in range(5):
            A[x][y] = B[x][y] ^ ((~B[(x + 1) % 5][y]) & B[(x + 2) % 5][y])
    #print("Результат чи\nA =", A)
    #printLine()
    return A

Шаг йота

In [11]:
def iota(argsA, iteration):
    argsA[0][0] ^= RC[iteration]
    #print("Результат йота\nA =", argsA)
    #printLine()
    return argsA

Функция перестановки

In [12]:
def f(A, iterations_number=24):
    for iteration in range(0, iterations_number):
        #print("раунд", iteration)
        #print("A = ", A)
        A = iota(
            chi(
                rho_pi(
                    theta(A)
                )
            ),
            iteration
        )

    return A

Отжимка и возврат

![title](img/img_1.png)

In [13]:
state = []
for blockId in range(0, blocksTotal):
    blockBytes = BAinput[blockId * (r // 8): (blockId + 1) * (r // 8)]
    block = []
    for start in range(0, r // 8, 8):
        block.append(int.from_bytes(blockBytes[start: start + 8], "little", signed=False))
    for start in range(r // 8, b // 8, 8):
        block.append(0)
    block = [block[i:21 + i:5] for i in range(0, 5)]
    if blockId != 0:
        for i in range(5):
            for j in range(5):
                state[i][j] ^= block[i][j]
    else:
        state = block
    state = f(state)

res = 0
for i in range(4):
    res |= state[i][0] << (i * 64)

print("SHA3-256:" + res.to_bytes(32, "little").hex())

SHA3-256:0469867610a7b89e0bb133ceffff1307d54d135b01a719ac689100fd05229421
